In [62]:
!pip install langchain-google-genai python-dotenv pandas

In [63]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

# Import necessary classes from langchain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

# Load environment variables from the .env file
load_dotenv()

True

In [64]:
# Get the Google API key from environment variables
KEY = os.getenv("GOOGLE_API_KEY")

# Initialize the Gemini model (using latest available model)
llm = ChatGoogleGenerativeAI(
    google_api_key=KEY,
    model="gemini-pro-latest",  # Updated to use latest model
    temperature=0.5
)

E0000 00:00:1759486197.726599   89782 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [65]:
llm

ChatGoogleGenerativeAI(model='models/gemini-pro-latest', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1144ad180>, default_metadata=())

In [66]:
# Define the desired JSON format for the quiz output.
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [67]:
# Create the prompt template for quiz generation.
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [68]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [69]:
# Create the LLMChain for generating the quiz.
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [70]:
# Create the prompt template for evaluating the generated quiz.
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [71]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [72]:
# Create the LLMChain for reviewing the quiz.
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [73]:
# Create the final SequentialChain that runs the quiz and review chains in order.
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [74]:
import json
import traceback

# This part of your code creates and then reads the data from data.txt
# For a real application, you would typically just have the reading part.
try:
    with open("data.txt", 'r') as file:
        TEXT = file.read()
except FileNotFoundError:
    print("Error: data.txt not found. Please make sure the file exists.")
    TEXT = None

if TEXT:
    # Define the input variables for the chain
    NUMBER = 5 
    SUBJECT = "Machine Learning"
    TONE = "simple"

    # This is the main execution step
    try:
        # The chain runs, and its entire output is stored in the 'response' variable
        response = generate_evaluate_chain(
            {
                "text": TEXT,
                "number": NUMBER,
                "subject": SUBJECT,
                "tone": TONE,
                "response_json": json.dumps(RESPONSE_JSON)
            }
        )
    except Exception as e:
        print("An error occurred while running the chain.")
        traceback.print_exc()
        response = None

# After running, 'response' will be a dictionary containing the results
if response:
    print("Successfully received response from the chain.")
    # print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

In [75]:
if response:
    # Use the .get() method to safely extract the quiz
    quiz = response.get("quiz")

    # Now the 'quiz' variable holds the JSON string of your MCQs
    print(quiz)

```json
{
  "1": {
    "mcq": "The early development of machine learning was inspired by human biology. According to the text, which psychologist introduced a theory about how interactions between nerve cells could form a basis for learning?",
    "options": {
      "a": "Arthur Samuel",
      "b": "Walter Pitts",
      "c": "Donald Hebb",
      "d": "Tom M. Mitchell"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "An early 'learning machine' called Cybertron was developed in the 1960s. Which of the following biological patterns was it trained to analyze?",
    "options": {
      "a": "DNA sequences",
      "b": "Electrocardiograms",
      "c": "Cell division rates",
      "d": "Brain wave patterns"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "According to the formal definition by Tom M. Mitchell, a computer program is said to learn if its performance at a task improves with what?",
    "options": {
      "a": "More complex algorithms",
      "b": "Faster computer hardware",

In [76]:
# Debug: Check the type and content of quiz before parsing
print(f"Type of quiz: {type(quiz)}")
print(f"First 200 characters of quiz: {str(quiz)[:200]}")

# Try to parse the JSON
try:
    if isinstance(quiz, str):
        quiz = json.loads(quiz)
        print("✅ Successfully parsed JSON")
    else:
        print("Quiz is already a dictionary")
except json.JSONDecodeError as e:
    print(f"❌ JSON parsing error: {e}")
    # Try to clean the JSON string if it has markdown formatting
    if quiz.startswith("```json"):
        # Remove markdown formatting
        quiz = quiz.split("```json")[1].split("```")[0].strip()
        quiz = json.loads(quiz)
        print("✅ Successfully parsed cleaned JSON")
    else:
        raise e

Type of quiz: <class 'str'>
First 200 characters of quiz: ```json
{
  "1": {
    "mcq": "The early development of machine learning was inspired by human biology. According to the text, which psychologist introduced a theory about how interactions between ner
❌ JSON parsing error: Expecting value: line 1 column 1 (char 0)
✅ Successfully parsed cleaned JSON


In [77]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [78]:
# Display the quiz in a nice format
print("="*80)
print("GENERATED MCQ QUIZ")
print("="*80)

for i, (key, value) in enumerate(quiz.items(), 1):
    print(f"\nQuestion {i}: {value['mcq']}")
    print("\nOptions:")
    for option, option_text in value['options'].items():
        marker = "✓" if option == value['correct'] else " "
        print(f"  {marker} {option}) {option_text}")
    print(f"\nCorrect Answer: {value['correct']}")
    print("-" * 60)


GENERATED MCQ QUIZ

Question 1: The early development of machine learning was inspired by human biology. According to the text, which psychologist introduced a theory about how interactions between nerve cells could form a basis for learning?

Options:
    a) Arthur Samuel
    b) Walter Pitts
  ✓ c) Donald Hebb
    d) Tom M. Mitchell

Correct Answer: c
------------------------------------------------------------

Question 2: An early 'learning machine' called Cybertron was developed in the 1960s. Which of the following biological patterns was it trained to analyze?

Options:
    a) DNA sequences
  ✓ b) Electrocardiograms
    c) Cell division rates
    d) Brain wave patterns

Correct Answer: b
------------------------------------------------------------

Question 3: According to the formal definition by Tom M. Mitchell, a computer program is said to learn if its performance at a task improves with what?

Options:
    a) More complex algorithms
    b) Faster computer hardware
    c) Huma

In [79]:
# Create a pandas DataFrame for better visualization
quiz_df = pd.DataFrame(quiz_table_data)
print("\nQuiz DataFrame:")
print(quiz_df.to_string(index=False))

# Also save to CSV
quiz_df.to_csv("generated_quiz.csv", index=False)
print(f"\n✅ Quiz saved to 'generated_quiz.csv'")



Quiz DataFrame:
                                                                                                                                                                                                               MCQ                                                                                              Choices Correct
The early development of machine learning was inspired by human biology. According to the text, which psychologist introduced a theory about how interactions between nerve cells could form a basis for learning?                             a: Arthur Samuel | b: Walter Pitts | c: Donald Hebb | d: Tom M. Mitchell       c
                                                                    An early 'learning machine' called Cybertron was developed in the 1960s. Which of the following biological patterns was it trained to analyze?           a: DNA sequences | b: Electrocardiograms | c: Cell division rates | d: Brain wave patterns       b
                       

In [80]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'10_03_2025_15_41_55'